In [1]:
#imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.signal as signal #signal processing stuff (e.g. filters, hilbert transform, etc.)
from scipy.stats import linregress
import scipy
import struct
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import svgutils.transform as sg
import sys

#nelpy!
import nelpy as nel  #recommended import for nelpy
import nelpy.plotting as npl  #recommended import for the nelpy plotting library
import nelpy.io.trodes as neltro

#I saved my online ripple detection filter taps and stuff in a file....gonna import this for all analysis
import imp
ripple_filtering = imp.load_source('ripple_filtering', '/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/DataAnalysisScripts/ripple_filtering.py')

/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#paper plot settings
sns.set(rc={'figure.figsize': (12, 4),'lines.linewidth': 1, 'font.size': 18, 'axes.labelsize': 16, 'axes.titlesize':18, 'legend.fontsize': 12, 'ytick.labelsize': 12, 'xtick.labelsize': 12 })
sns.set_style('white')
sns.set_color_codes(palette='colorblind')
#plots show up within jupyter for matplotlib
%matplotlib inline 

# Load and save data into txt files

In [3]:
FS=3000
FS_system = 30000
#din 11 for tetrode 3 channel 1 recordings

#Load Timestamps
filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.timestamps.dat"
print("*****************Loading LFP Timestamps*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
timeStamps = np.fromfile(f, dtype=np.uint32)
timeStampsSeconds = (timeStamps-timeStamps[0])/FS_system
print("Done!")

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt2ch1.dat" 
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
data = np.fromfile(f, dtype=np.int16)

start = 0
while(timeStamps[start]%10 != 0):
    start+=1
timeStamps = timeStamps[start::10]
data = data[start::10]
timeStampsSeconds = timeStampsSeconds[start::10]
print(timeStamps[0])
print("Data decimated to match data sent to modules")

*****************Loading LFP Timestamps*****************
b'<Start settings>\n'
b'Description: LFP timestamps\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'Clock rate: 30000\n'
b'Decimation: 1\n'
b'Time_offset: 0\n'
b'Fields: <time uint32>\n'
Current file position 185
Done!
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'nTrode_ID: 2\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Decimation: 1\n'
b'First_timestamp: 19935\n'
b'Reference: on\n'
b'Low_pass_filter: 400\n'
b'Fields: <voltage int16>\n'
Current file position 294
19940
Data decimated to match data sent to modules


In [4]:
(timeStampsSeconds[-1]-timeStampsSeconds[0])/60

80.04678333333332

In [7]:
np.savetxt('/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/dataT2Full.out',data)
np.savetxt('/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/timeStampsSeconds.out',timeStampsSeconds)

In [7]:
#detect cannonical ripples
smoothed_envelope, rippleData = ripple_filtering.rippleBandFilter(data, timeStampsSeconds, FS=FS)
#set threshold, find threshold crossings, and find ripple bounds
SecondaryThresholdSigma = 0
thresholdSigma = 3
lengthCriteria = 0.015 #15 ms
#Tetrode 2
threshold = np.mean(smoothed_envelope) + thresholdSigma*np.std(smoothed_envelope)

#rippleEvents contains indices of events in smoothed_envelope that meet threshold
rippleEvents, _ = ripple_filtering.thresholdCrossings(smoothed_envelope, threshold)
#remove events that don't meet length criteria
rippleEvents = rippleEvents[rippleEvents[:,1] - rippleEvents[:,0] >= np.round(FS * lengthCriteria),:]

#find start and end of putative ripple aka when ripple band signal comes back down to mean
secondaryThreshold = np.mean(smoothed_envelope) + SecondaryThresholdSigma*np.std(smoothed_envelope)
rippleBounds, broaderMaxes = ripple_filtering.thresholdCrossings(smoothed_envelope, secondaryThreshold)

#find windows that match and where they match aka where in the larger bound window does the smaller bound fit
outerBoundaryIndices = np.searchsorted(rippleBounds[:,0], rippleEvents[:,0])
outerBoundaryIndices = outerBoundaryIndices - 1 #subtract 1 since searchsorted returns index after where it belongs

#Find extended boundaries for ripple events by pairing to largere window
#   (Note that there may be repeats if the larger window contains multiple > 3SD sections)
rippleBounds = rippleBounds[outerBoundaryIndices,:]
rippleMaxes = broaderMaxes[outerBoundaryIndices]

# Now, since all that we care about are the larger windows, so we should get rid of repeats
_, unique_idx = np.unique(rippleBounds[:,0], return_index=True)
rippleBounds = rippleBounds[unique_idx,:]
rippleMaxes = rippleMaxes[unique_idx]
rippleEvents = rippleEvents[unique_idx,:]

/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/signaltools.py:1333: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/signaltools.py:1336: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array

In [6]:
np.savez("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/smoothed_envelope25tap.npz",smoothed_envelope=smoothed_envelope)

In [8]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsStart3SD.out",rippleBounds[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsEnd3SD.out",rippleBounds[:,1],fmt='%10.5f')

In [5]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsStart5SD.out",rippleBounds[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsEnd5SD.out",rippleBounds[:,1],fmt='%10.5f')

In [6]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsStart4SD.out",rippleBounds[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsEnd4SD.out",rippleBounds[:,1],fmt='%10.5f')

In [11]:
ripples4SD = np.loadtxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/rippleBoundsEnd4SD.out")
ripples4SD.size

1041

In [6]:
#save data for simulated detections
smoothed_envelope_simulated, rippleData_simulated = ripple_filtering.rippleBandFilterSimulated(data, timeStampsSeconds, FS, ripple_filtering.bandpassFilterTaps, ripple_filtering.lowpassFilterTaps)
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChanAnalysis/smoothed_envelope_simulated.out",smoothed_envelope_simulated,fmt='%10.5f')

# Call C++ analysis program
Remember to compile before running it! Compile with below!<br>
` g++ -o singleChanAnalysis -Wall -pedantic -std=c++11 -pthread main.cpp`

In [9]:
os.system("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/DataAnalysisScripts/singleChannelSimulatedDetections/singleChanAnalysis")

0

# Display results!
Meh...this is in the Figure 5 notebook